In [179]:
import re
from collections import Counter
import pymorphy2
from tqdm import tqdm
import os
import pandas as pd
import math
import nltk
import numpy as np

In [66]:
marked_path = '/home/nst/mount/data/share/yd/popular_science_texts_store/ner_markup/final_markup/'

In [67]:
def slurp(path):
    with open(path, 'r') as file_object:
        return file_object.read()

In [68]:
def extract_ne(text):
    pattern = re.compile(r'&(.*?)!&')
    nes = re.findall(pattern, text)
    lemmas = []
    for ne in nes:
        divided = ne.split()
        lemma = [morph.parse(word)[0].normal_form for word in divided]
        lemma = ' '.join(lemma)
        lemmas.append(lemma)
    return lemmas

In [69]:
def range_ne(texts_path, top_number):
    nes = []
    texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            texts.append(marked_text)
    for text in tqdm(texts):
        found_ne = extract_ne(text)
        #print(found_ne)
        nes.extend(found_ne)
    ne_dict = Counter(nes)
    ranged_ne = list(sorted(ne_dict, key = lambda x : x[1], reverse=True))
    print(top_number, "most cited scholars are:\n", ranged_ne[:top_number]) 
    return ranged_ne

## Get NE context

In [173]:
class Context:
    def __init__(self, left_context: str, right_context: str,
                 left_word: str, word: str):
        self.left_context = left_context
        self.right_context = right_context
        self.left_word = left_word
        self.word = word
    def __repr__(self):
        return 'Context("%s", "%s", "%s", "%s")' % (
            self.left_context, self.right_context,
            self.left_word, self.word)

def tokenize_text(text: str) -> [str]:
    return re.findall(r'&?[\w.\'-]+!?&?', text, flags=re.UNICODE)

def cleanup_tag(text: str) -> str:
    return text.replace('&', '').replace('!', '')

def get_span(tokens: [str], start: int, end: int) -> str:
    return cleanup_tag(' '.join(tokens[start:end]))

def get_complete_word(index: int, tokens: [str]) -> (str, int, bool):
    current_word = tokens[index]
    if not current_word.startswith('&'):
        return current_word, index+1, False

    word_parts = [current_word]
    index += 1
    if not current_word.endswith('!&'):
        while index < len(tokens):
            current_word = tokens[index]
            word_parts.append(current_word)
            if current_word.endswith('!&'):
                index += 1
                break
            index += 1
        else:
            raise ValueError('No matching closing tag in tokens: "%s"' % tokens)
    
    word = cleanup_tag(' '.join(word_parts))
    return word, index, True

assert ('a', 1, False) == get_complete_word(0, ['a', 'b'])
assert ('b', 2, False) == get_complete_word(1, ['a', 'b'])
assert ('a', 1, True) == get_complete_word(0, ['&a!&', 'b']), get_complete_word(0, ['&a!&', 'b'])
assert ('a b', 2, True) == get_complete_word(0, ['&a', 'b!&'])
assert ('a b c', 3, True) == get_complete_word(0, ['&a', 'b', 'c!&'])

def extract_contexts(text: str, window_size: int) -> [Context]:
    result = []
    
    tokens = tokenize_text(text)
    index = 0

    while index < len(tokens):
        word, new_index, is_tag = get_complete_word(index, tokens)
        if not is_tag:
            index = new_index
            continue

        left_context = get_span(tokens, index-5, index)
        right_context = get_span(tokens, new_index+1, new_index+6)
        left_word = get_span(tokens, index-1, index)
        
        context = Context(left_context, right_context, left_word, word)
        result.append(context)
        
        index = new_index
    
    return result

print(extract_contexts(text_3, 5))

[Context("вполне серьезная книга. Есть книга", "язык на грани нервного срыва", "книга", "Максима Кронгауза"), Context("второе переработанное более полное издание.", "тоже выпускник нашего отделения.", "издание.", "Кронгауз")]


In [102]:
text_3 = '«Русский со словарем» — очень забавная, изящная, но вполне серьезная книга. Есть книга &Максима Кронгауза!& «Русский язык на грани нервного срыва», у которой уже второе, переработанное, более полное издание. &Кронгауз!&, кстати, тоже выпускник нашего отделения.'

In [92]:
get_context(preprocessed_texts.preprocessed_texts[1], indices, 5)

(['как экспериментальное ответвление генеративной грамматики'],
 ['Н. Хомского '],
 ['и стала известна как формальная психолингвистика'])

## Make Dataframe and get all NE contexts

### Marked & Preprocessed DataFrame

In [182]:
def make_texts_df(texts_path):
    marked_texts = []
    for root, dirs, files in os.walk(texts_path):
        for file_name in files:
            input_path = texts_path + file_name
            marked_text = slurp(input_path)
            marked_texts.append(marked_text)
    marked_df = pd.DataFrame(marked_texts, columns=['marked_texts'])
    return marked_df

In [183]:
marked_df = make_texts_df(marked_path)

In [184]:
marked_df.head()

,marked_texts
0,Как интересно сгруппированы атомы... комплимен...
1,Что люди обычно думают о теоретической лингвис...
2,Радиопульсары – одно из наблюдательных проявле...
3,"Мне уже много лет. Когда я заканчивал школу, э..."
4,Совместно с издательством «Альпина нон-фикшн» ...


### Get all contexts from marked texts with window 5

In [181]:
def compile_contexts_dataframe(texts, window_size):
    left = []
    left_words = []
    entities = []
    right = []
    for text in texts:
        contexts = extract_contexts(text, window_size)
        for context in contexts:
            left_context = context.left_context
            left.append(left_context)
            left_word = context.left_word
            left_words.append(left_word)
            entity = context.word
            entities.append(entity)
            right_context = context.right_context
            right.append(right_context)
    dataframe = pd.DataFrame(np.column_stack([left, left_words, entities, right]),
                            columns=['left_context', 'left_word', 
                                     'named_entity', 'right_context'])
    
    return dataframe
    

In [196]:
def sort_dataframe(dataframe, column):
    return dataframe.sort_values(by=[column], axis=0)

In [194]:
contexts_df = compile_contexts_dataframe(marked_df.marked_texts, 5)

In [195]:
contexts_df

,left_context,left_word,named_entity,right_context
0,Из выступления лауреата Нобелевской премии,премии,Ричарда Фейнмана,Калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,году,Гердом Биннигом,Генрихом Рорером сотрудниками швейцарского отд...
2,1981 году Гердом Биннигом и,и,Генрихом Рорером,швейцарского отделения IBM сконструировавшими ...
3,сотрудниками IBM Almaden Research Center,Center,Дональдом Эйглером,Эрхардом Швейцером в 1990 году
4,Research Center Дональдом Эйглером и,и,Эрхардом Швейцером,1990 году заключался в том
5,подобная работа была проделана группой,группой,Джозефа Стросцио,Национального Института Стандартов и Технологий
6,решётки из атомов подложки Когда,Когда,Дональд Эйглер,назад выступал с лекцией в
7,этом больше всех писал академик,академик,Андрей Анатольевич Зализняк,Дело в том что написать
8,лингвистике. Это прежде всего книга,книга,Зализняка,заметок о любительской лингвистике книга
9,заметок о любительской лингвистике книга,книга,Ирины Левонтиной,со словарем очень забавная изящная


In [198]:
sorted_df = sort_dataframe(contexts_df, 'left_word' )

In [202]:
sorted_df = sorted_df.drop_duplicates()

In [203]:
sorted_df.to_csv('ne_contexts_sorted.tsv', sep='\t')

In [224]:
remove_duplicates = contexts_df.drop_duplicates()

In [225]:
len(remove_duplicates)

1255

In [227]:
remove_duplicates = remove_duplicates.reset_index(drop=True)
remove_duplicates.head()

,left_context,named_entities,right_context
0,Из выступления лауреата Нобелевской премии,Ричарда Фейнмана,в Калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,Гердом Биннигом,и Генрихом Рорером сотрудниками швейцарского о...
2,1981 году Гердом Биннигом и,Генрихом Рорером,сотрудниками швейцарского отделения IBM сконст...
3,сотрудниками IBM Almaden Research Center,Дональдом Эйглером,и Эрхардом Швейцером в 1990 году
4,Research Center Дональдом Эйглером и,Эрхардом Швейцером,в 1990 году заключался в том


In [228]:
remove_duplicates.to_csv('contexts_draft.tsv', sep='\t')

In [101]:
for root, dirs, files in os.walk(marked_path):
    for file_name in files:
        input_path = marked_path + file_name
        marked_text = slurp(input_path)
        if 'Кронгауз' in marked_text:
            print(file_name)

126.txt


In [160]:
'Когда &Гарри Уиттингтон!& и его ученики &Дерек Бриггс!& и &Саймон Конвей Моррис!&'

'Когда &Гарри Уиттингтон!& и его ученики &Дерек Бриггс!& и &Саймон Конвей Моррис!&'

In [93]:
df = pd.read_csv('/home/nst/mount/data/linguistics_hse/popular-science-research/popular-science-repo/Popular-Science-Texts-Compling-research/ner_markup/contexts_draft.tsv', sep='\t')

In [94]:
df

,Unnamed: 0,left_context,named_entities,right_context
0,0,Из выступления лауреата Нобелевской премии,Ричарда Фейнмана,в Калифорнийском технологическом институте в 1959
1,1,проблема была в 1981 году,Гердом Биннигом,и Генрихом Рорером сотрудниками швейцарского о...
2,2,1981 году Гердом Биннигом и,Генрихом Рорером,сотрудниками швейцарского отделения IBM сконст...
3,3,сотрудниками IBM Almaden Research Center,Дональдом Эйглером,и Эрхардом Швейцером в 1990 году
4,4,Research Center Дональдом Эйглером и,Эрхардом Швейцером,в 1990 году заключался в том
5,5,подобная работа была проделана группой,Джозефа Стросцио,из Национального Института Стандартов и Технол...
6,6,сотрудниками IBM Almaden Research Center,Дональдом Эйглеро,м и Эрхардом Швейцером в 1990
7,7,этом больше всех писал академик,Андрей Анатольевич Зализняк,. Дело в том что написать
8,8,лингвистике. Это прежде всего книга,Зализняка,Из заметок о любительской лингвистике книга
9,9,заметок о любительской лингвистике книга,Ирины Левонтиной,Русский со словарем очень забавная изящная


## Count most frequent collocations in contexts

In [33]:
def preprocess(texts):
    preproc_texts = []
    pattern = re.compile(r'\w+')
    try:
        for text in tqdm(texts):
            text = text.lower()
            words = ' '.join(re.findall(pattern, text))
            preproc_texts.append(words)
    except AttributeError:
        print('Corrupted:', text)
        preproc_texts.append('')
    return preproc_texts

In [37]:
preprocessed_left = preprocess(df.left_context)
preprocessed_right = preprocess(df.right_context)

100%|██████████| 1216/1216 [00:00<00:00, 93964.03it/s]


In [36]:
df = df.dropna(axis=0, how='any')
len(df)

1216

In [38]:
preprocessed_df = pd.DataFrame(preprocessed_left, columns=['left'])
preprocessed_df['right'] = preprocessed_right
preprocessed_df.head(3)

,left,right
0,из выступления лауреата нобелевской премии,в калифорнийском технологическом институте в 1959
1,проблема была в 1981 году,и генрихом рорером сотрудниками швейцарского о...
2,1981 году гердом биннигом и,сотрудниками швейцарского отделения ibm сконст...


In [44]:
def compile_tokens_list(texts):
    tokens = []
    for text in texts:
        text = text.split()
    tokens.extend(text)
    tokens = ' '.join(tokens)
    return tokens

In [45]:
tokens_left = compile_tokens_list(preprocessed_df.left)
tokens_right = compile_tokens_list(preprocessed_df.right)

In [47]:
def aprior_probability(dictionary):
    s = sum(dictionary.values())
    for word  in dictionary:
        dictionary[word] /= s
   # aprior_dictionary = dict(wiki_freq).get(word)/s
    return dict(dictionary)

In [48]:
def bigram_probability(bigrams):
    count_bigrams = len(bigrams)
    joined_bigrams = [' '.join(bigram) for bigram in bigrams]
    bigrams_dict = Counter(joined_bigrams)
    for bigram in bigrams_dict:
        bigrams_dict[bigram] /= count_bigrams
    return dict(bigrams_dict)

In [49]:
def count_pmi(bigrams, bigrams_prob, words_aprior):
    joined_bigrams = []
    pmis = []
    for bigram in bigrams:
        #print(bigram)
        joined_bigram = ' '.join(bigram)
        pmi = math.log(bigrams_prob.get(joined_bigram)/
                      (words_aprior.get(bigram[0])*words_aprior.get(bigram[1])))
        joined_bigrams.append(joined_bigram)
        pmis.append(pmi)
    counted_pmi = list(zip(joined_bigrams, pmis))
    return counted_pmi

In [51]:
def range_pmi(pmis_list, top_k):
    pmis_dict = dict(pmis_list)
    best_pmis = sorted(pmis_dict, key = lambda x:x[1], reverse = True)
    print(best_pmis[:top_k])
    return best_pmis

In [52]:
def extract_collocations(tokens, top_best):
    freq = Counter(tokens.split())
    bigrams = list(nltk.bigrams(tokens.split()))
    aprior = aprior_probability(freq)
    bigrams_prob = bigram_probability(bigrams)
    pmi = count_pmi(bigrams, bigrams_prob, aprior)
    best = range_pmi(pmi, top_best)
    return best

In [56]:
# top best left 
collocations_left = extract_collocations(tokens_left, 10)

['мы рассказывали', 'совсем недавно', 'недавно мы', 'рассказывали о']


In [57]:
# top best right
collocations_right = extract_collocations(tokens_right, 10)

['истории об', 'архивной находке', 'его истории', 'об архивной', 'и его']


In [65]:
df.left_context[200:]

210     соотечественником Виталием Гинзбургом и америк...
211     интервью рассказывает доктор психологических наук
212          годов которые проводили выдающиеся археологи
213            выдающиеся археологи Артемий Арциховский и
214            Впрочем позже другие археологи петербуржец
215                      культ которого установил на Руси
216                                  . Однако в 1939 году
217                        1939 году Владимир Богусевич и
218                      рубрике Как получить Нобелевку .
219                     И со стипендией которая позволила
220            нобелиата началась с Джей-Джея знаменитого
222                     получить Нобелевку . Оуэн Уилланс
223             с другими студентами будущими нобелиатами
224     студентами будущими нобелиатами Эрнестом Резер...
225     Резерфордом Чарльзом Вильсоном будущим доктора...
227                         потом и любовником Марии Кюри
228                 с Джей-Джея знаменитого Джозефа Джона
229           